This script opens geojson files for different communities that outline the neighbourhood boundaries and consolidates them into a single database that can be used to generate folium maps, or to create new geojson files with additional information folded in

In [ ]:
import json
from tinydb import TinyDB, Query

# Explore files and their structure

In [ ]:
%ls

### Cambridge

In [ ]:
with open('Cambridge.geojson') as Cambridge:
    Cambridge_data = json.load(Cambridge)

In [ ]:
len(Cambridge_data['features'])

In [ ]:
Cambridge_data['features'][0].keys()

In [ ]:
for feature in Cambridge_data['features']:
    print('geometry keys {}'.format(feature['geometry'].keys()))
    print('properties keys {}'.format(feature['properties']))

### Kitchener

In [ ]:
with open('Kitchener.geojson') as Kitchener:
    Kitchener_data = json.load(Kitchener)

In [ ]:
len(Kitchener_data['features'])

In [ ]:
Kitchener_data['features'][0].keys()

In [ ]:
for feature in Kitchener_data['features']:
    print('geometry keys {}'.format(feature['geometry'].keys()))
    print('properties keys {}'.format(feature['properties']))

### Waterloo

In [ ]:
with open('Waterloo.geojson') as Waterloo:
    Waterloo_data = json.load(Waterloo)

In [ ]:
len(Waterloo_data['features'])

In [ ]:
Waterloo_data['features'][0].keys()

In [ ]:
for feature in Waterloo_data['features']:
    print('geometry keys {}'.format(feature['geometry'].keys()))
    print('properties keys {}'.format(feature['properties']))
    print('coordinates type {}'.format(type(feature['geometry']['coordinates'])))

In [ ]:
Waterloo_data['features'][0]['geometry']['coordinates'][0]

# Create a tinydb as a simple way of storing the data we need for future reference

In [ ]:
# open a tiny db to stow away all of the neighbourhood data in a consistent format
# Instantiate the database and query object
db = TinyDB('Waterloo_Kitchener_Cambridge.json')
representative = Query()

### extract the key data features we need, namely Neighbourhood label and geometry with the aid of a helper function

In [ ]:
def extract_data_features(json_file, hood_label, city, tdb, tdb_query):
    '''
    param: json_file = the geojson file for one of the cities
    param: hood_label = the name used to label the neighbourhood
           kitchener = 'PLANNING_C'
           waterloo = 'DISTNAME'
           cambridge = 'NHOOD_NAME'
    param: tdb = tinydb object already instantiated
    param: tdb_query = tinydb Query() object
    '''
    with open(json_file) as f:
        data = json.load(f)
    
    for hood in data['features']:
        hood_name = hood['properties'][hood_label] # name of neighbourhood
        raw_coords = hood['geometry']['coordinates'][0]
        coords = [[x[1], x[0]] for x in raw_coords]

        if not tdb.search((tdb_query.name == hood_name) & (tdb_query.city == city)):
            print('adding {} from {} to database'.format(hood_name, city))
            tdb.insert({'name': hood_name,
                           'city': city,
                           'coords': coords})
        else:
            print('previously logged {} from {}'.format(hood_name, city))

In [ ]:
extract_data_features('Waterloo.geojson', 'DISTNAME', 'Waterloo', db, representative)

In [ ]:
extract_data_features('Kitchener.geojson', 'PLANNING_C', 'Kitchener', db, representative)

In [ ]:
extract_data_features('Cambridge.geojson', 'NHOOD_NAME', 'Cambridge', db, representative)

In [ ]:
db.close()